# **Food Management System**: Analysis with SQL and Python

Project Type:  Data Analysis and System Performance with Sql.


---

Contribution: Individual (Ashutosh Santosh Rane)



Problem Statement:

Food wastage is a significant issue, with many households and restaurants discarding surplus food while numerous people struggle with food insecurity. This project aims to develop a Local Food Wastage Management System, where:
Restaurants and individuals can list surplus food.
NGOs or individuals in need can claim the food.
SQL stores available food details and locations.
A Streamlit app enables interaction, filtering, CRUD operation and visualization.

Business Use Cases
Connecting surplus food providers to those in need through a structured platform.
Reducing food waste by redistributing excess food efficiently.
Enhancing accessibility via geolocation features to locate food easily.
Data analysis on food wastage trends for better decision-making.

Approach
 1. Data Preparation
Utilize a provided dataset containing food donation records.
Ensure consistency and accuracy in data formatting.
2. Database Creation
Store food availability data in SQL tables.
Implement CRUD operations for updating, adding, and removing records.
3. Data Analysis.
Identify food wastage trends based on categories, locations, and expiry dates.
Generate reports for effective food distribution.
4. Application Development
Develop a Streamlit-based user interface to:
Display output of the  15 SQL queries .
Provide filtering options based on city, provider, food type, and meal type.
Show contact details of providers for direct coordination.
5. Deployment
Deploy the Streamlit application for accessibility and real-time interaction

Data Flow and Architecture
1. Data Storage:
Use SQL database to store food donations, locations, and provider details.
2. Processing Pipeline:
Do analysis and generate insights into food wastage patterns.
3. Deployment:
Develop a Streamlit-based interface for food providers and seekers.


In [ ]:
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_db_connection():
    """Establish connection to SQLite database"""
    try:
        conn = sqlite3.connect('food_donation.db')
        print("Database connection established successfully!")
        return conn
    except Exception as e:
        print(f" Database connection failed: {e}")
        return None

In [ ]:
conn = get_db_connection()

Database connection established successfully!


In [ ]:
def execute_query(query, description=""):
    """Execute SQL query and return results as DataFrame"""
    try:
        df = pd.read_sql_query(query, conn)
        if description:
            print(f"\n{description}")
            print("-" * 50)
        return df
    except Exception as e:
        print(f" Query execution failed: {e}")
        return pd.DataFrame()

In [ ]:
def display_results(df, title=""):
    """Display query results"""
    if df.empty:
        print("No data found.")
        return

    print(f"\n{title}")
    print(df.to_string(index=False))
    print(f"\nTotal records: {len(df)}")

In [ ]:
# QUERY 1: Food providers and receivers in each city

query1 = """
SELECT
    City,
    SUM(CASE WHEN Type = 'Provider' THEN Count ELSE 0 END) as Providers,
    SUM(CASE WHEN Type = 'Receiver' THEN Count ELSE 0 END) as Receivers
FROM (
    SELECT City, COUNT(*) as Count, 'Provider' as Type FROM providers GROUP BY City
    UNION ALL
    SELECT City, COUNT(*) as Count, 'Receiver' as Type FROM receivers GROUP BY City
) combined
GROUP BY City
ORDER BY Providers DESC, Receivers DESC;
"""

result1 = execute_query(query1, "QUERY 1: Food providers and receivers in each city")
display_results(result1, "Providers and Receivers by City")


QUERY 1: Food providers and receivers in each city
--------------------------------------------------

Providers and Receivers by City
                    City  Providers  Receivers
               New Carol          3          0
South Christopherborough          3          0
            Lake Michael          2          1
              New Daniel          2          1
          North Michelle          2          1
            Port Melissa          2          1
        West Christopher          2          1
             Bradleyport          2          0
              Davidville          2          0
            East Anthony          2          0
           East Jennifer          2          0
               East Lisa          2          0
            East Melissa          2          0
          East Stephanie          2          0
               Jamesport          2          0
            Jonathanstad          2          0
           Lake Benjamin          2          0
               Lak

In [ ]:
# QUERY 2: Which type of food provider contributes the most food?

query2 = """
SELECT
    p.Type as Provider_Type,
    COUNT(fl.Food_ID) as Total_Food_Listings,
    SUM(fl.Quantity) as Total_Food_Quantity,
    AVG(fl.Quantity) as Average_Quantity_Per_Listing
FROM providers p
LEFT JOIN food_listings fl ON p.Provider_ID = fl.Provider_ID
GROUP BY p.Type
ORDER BY Total_Food_Quantity DESC;
"""

result2 = execute_query(query2, "QUERY 2: Food provider types and their contributions")
display_results(result2, " Provider Type Contributions")


QUERY 2: Food provider types and their contributions
--------------------------------------------------

 Provider Type Contributions
   Provider_Type  Total_Food_Listings  Total_Food_Quantity  Average_Quantity_Per_Listing
      Restaurant                  258                 6923                     26.833333
     Supermarket                  267                 6696                     25.078652
Catering Service                  232                 6116                     26.362069
   Grocery Store                  243                 6059                     24.934156

Total records: 4


In [ ]:
# QUERY 3: Contact information of food providers in a specific city

query3 = """
SELECT
    City,
    Name as Provider_Name,
    Type as Provider_Type,
    Address,
    Contact,
    COUNT(fl.Food_ID) as Active_Listings
FROM providers p
LEFT JOIN food_listings fl ON p.Provider_ID = fl.Provider_ID
GROUP BY p.Provider_ID, p.City, p.Name, p.Type, p.Address, p.Contact
ORDER BY City, Active_Listings DESC;
"""

result3 = execute_query(query3, "QUERY 3: Provider contact information by city")
display_results(result3, " Provider Contact Information")


QUERY 3: Provider contact information by city
--------------------------------------------------

 Provider Contact Information
                    City                     Provider_Name    Provider_Type                                                       Address                Contact  Active_Listings
                Adambury                        Ibarra LLC Catering Service              064 Andrea Land Suite 946\nLake Melody, ME 49581             6703380260                1
               Adamsview                     Lozano-Miller       Restaurant           590 Michelle Brooks Apt. 406\nDanielburgh, TN 15283       001-281-026-8022                0
              Adamsville                         Davis Ltd       Restaurant          88279 Luis Throughway Apt. 639\nThomasberg, NM 37613      (112)122-3591x558                0
            Aguirreville                         Rowe-Chen      Supermarket                       304 Leslie Key\nMiddletonfurt, IN 01285             822889124

In [ ]:
# QUERY 4: Which receivers have claimed the most food?

query4 = """
SELECT
    r.Name as Receiver_Name,
    r.Type as Receiver_Type,
    r.City,
    COUNT(c.Claim_ID) as Total_Claims,
    COUNT(CASE WHEN c.Status = 'Completed' THEN 1 END) as Completed_Claims,
    COUNT(CASE WHEN c.Status = 'Pending' THEN 1 END) as Pending_Claims,
    SUM(fl.Quantity) as Total_Food_Quantity_Claimed
FROM receivers r
LEFT JOIN claims c ON r.Receiver_ID = c.Receiver_ID
LEFT JOIN food_listings fl ON c.Food_ID = fl.Food_ID
GROUP BY r.Receiver_ID, r.Name, r.Type, r.City
ORDER BY Total_Claims DESC
LIMIT 15;
"""

result4 = execute_query(query4, "QUERY 4: Top receivers by food claims")
display_results(result4, "Top Food Receivers")


QUERY 4: Top receivers by food claims
--------------------------------------------------

Top Food Receivers
    Receiver_Name Receiver_Type             City  Total_Claims  Completed_Claims  Pending_Claims  Total_Food_Quantity_Claimed
     Scott Hunter    Individual         Greenton             5                 1               0                          157
William Frederick           NGO        Port Dean             5                 0               1                          107
     Matthew Webb       Charity       West David             5                 2               1                          191
   Anthony Garcia    Individual        Brownbury             5                 1               2                          162
       Alvin West       Shelter        Kellybury             4                 2               1                           99
  Jennifer Nelson    Individual       North Paul             4                 0               2                          139
 Kristin

In [ ]:
# QUERY 5: Total quantity of food available from all providers.

query5 = """
SELECT
    SUM(Quantity) as Total_Food_Available,
    COUNT(*) as Total_Food_Listings,
    AVG(Quantity) as Average_Quantity_Per_Listing,
    MIN(Quantity) as Minimum_Quantity,
    MAX(Quantity) as Maximum_Quantity
FROM food_listings;
"""

result5 = execute_query(query5, "QUERY 5: Total food availability statistics")
display_results(result5, "Food Availability Overview")


QUERY 5: Total food availability statistics
--------------------------------------------------

Food Availability Overview
 Total_Food_Available  Total_Food_Listings  Average_Quantity_Per_Listing  Minimum_Quantity  Maximum_Quantity
                25794                 1000                        25.794                 1                50

Total records: 1


In [ ]:
# QUERY 6: Which city has the highest number of food listings?

query6 = """
SELECT
    Location as City,
    COUNT(*) as Food_Listings,
    SUM(Quantity) as Total_Food_Quantity,
    AVG(Quantity) as Average_Quantity,
    COUNT(DISTINCT Provider_ID) as Unique_Providers
FROM food_listings
GROUP BY Location
ORDER BY Food_Listings DESC;
"""

result6 = execute_query(query6, "QUERY 6: Food listings by city")
display_results(result6, "Cities by Food Listings")


QUERY 6: Food listings by city
--------------------------------------------------

Cities by Food Listings
                    City  Food_Listings  Total_Food_Quantity  Average_Quantity  Unique_Providers
           South Kathryn              6                  179         29.833333                 1
               New Carol              6                  167         27.833333                 2
               Perezport              5                  123         24.600000                 1
               Jimmyberg              5                  150         30.000000                 1
             East Angela              5                   90         18.000000                 1
       West Whitneymouth              4                   99         24.750000                 1
              West Kevin              4                   59         14.750000                 1
       South Thomasville              4                  111         27.750000                 1
South Christopherbo

In [ ]:
# QUERY 7: Most commonly available food types

query7 = """
SELECT
    Food_Type,
    COUNT(*) as Number_of_Listings,
    SUM(Quantity) as Total_Quantity,
    AVG(Quantity) as Average_Quantity,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM food_listings), 2) as Percentage_of_Total
FROM food_listings
GROUP BY Food_Type
ORDER BY Number_of_Listings DESC;
"""

result7 = execute_query(query7, "QUERY 7: Most common food types")
display_results(result7, "Food Types Distribution")


QUERY 7: Most common food types
--------------------------------------------------

Food Types Distribution
     Food_Type  Number_of_Listings  Total_Quantity  Average_Quantity  Percentage_of_Total
    Vegetarian                 336            8340         24.821429                 33.6
         Vegan                 334            8798         26.341317                 33.4
Non-Vegetarian                 330            8656         26.230303                 33.0

Total records: 3


In [ ]:
# QUERY 8: How many food claims have been made for each food item?

query8 = """
SELECT
    fl.Food_Name,
    fl.Food_Type,
    fl.Meal_Type,
    fl.Quantity as Available_Quantity,
    COUNT(c.Claim_ID) as Total_Claims,
    COUNT(CASE WHEN c.Status = 'Completed' THEN 1 END) as Completed_Claims,
    COUNT(CASE WHEN c.Status = 'Pending' THEN 1 END) as Pending_Claims,
    COUNT(CASE WHEN c.Status = 'Cancelled' THEN 1 END) as Cancelled_Claims,
    p.Name as Provider_Name
FROM food_listings fl
LEFT JOIN claims c ON fl.Food_ID = c.Food_ID
JOIN providers p ON fl.Provider_ID = p.Provider_ID
GROUP BY fl.Food_ID, fl.Food_Name, fl.Food_Type, fl.Meal_Type, fl.Quantity, p.Name
ORDER BY Total_Claims DESC
LIMIT 20;
"""

result8 = execute_query(query8, "QUERY 8: Food claims per item")
display_results(result8, "Claims per Food Item")



QUERY 8: Food claims per item
--------------------------------------------------

Claims per Food Item
 Food_Name      Food_Type Meal_Type  Available_Quantity  Total_Claims  Completed_Claims  Pending_Claims  Cancelled_Claims               Provider_Name
      Soup          Vegan     Lunch                  36             5                 3               1                 1               Rogers-Warren
   Chicken          Vegan    Snacks                   9             5                 1               1                 3    Phelps, Graham and Hayes
      Fish          Vegan    Snacks                  26             5                 2               3                 0                 Ramos Group
      Rice          Vegan Breakfast                   6             4                 0               2                 2                Williams-Cox
   Chicken     Vegetarian Breakfast                  42             4                 2               0                 2           Butler-Richard

In [ ]:
# QUERY 9: Provider with highest number of successful food claims

query9 = """
SELECT
    p.Name as Provider_Name,
    p.Type as Provider_Type,
    p.City,
    COUNT(fl.Food_ID) as Total_Food_Listings,
    COUNT(c.Claim_ID) as Total_Claims,
    COUNT(CASE WHEN c.Status = 'Completed' THEN 1 END) as Successful_Claims,
    ROUND(COUNT(CASE WHEN c.Status = 'Completed' THEN 1 END) * 100.0 / COUNT(c.Claim_ID), 2) as Success_Rate_Percentage,
    SUM(CASE WHEN c.Status = 'Completed' THEN fl.Quantity ELSE 0 END) as Total_Food_Distributed
FROM providers p
LEFT JOIN food_listings fl ON p.Provider_ID = fl.Provider_ID
LEFT JOIN claims c ON fl.Food_ID = c.Food_ID
GROUP BY p.Provider_ID, p.Name, p.Type, p.City
HAVING COUNT(c.Claim_ID) > 0
ORDER BY Successful_Claims DESC, Success_Rate_Percentage DESC
LIMIT 15;
"""

result9 = execute_query(query9, "QUERY 9: Providers with most successful claims")
display_results(result9, "Top Performing Providers")


QUERY 9: Providers with most successful claims
--------------------------------------------------

Top Performing Providers
              Provider_Name    Provider_Type             City  Total_Food_Listings  Total_Claims  Successful_Claims  Success_Rate_Percentage  Total_Food_Distributed
                Barry Group       Restaurant    South Kathryn                    9             7                  5                    71.43                     140
                 Miller Inc    Grocery Store        Coleburgh                    5             4                  4                   100.00                      94
  Barnes, Castro and Curtis       Restaurant   Zimmermanville                    6             5                  4                    80.00                      90
Harper, Blake and Alexander Catering Service       Devinmouth                    6             6                  4                    66.67                      56
          Butler-Richardson    Grocery Store East 

In [ ]:
# QUERY 10: Percentage of food claims (completed vs pending vs canceled)

query10 = """
SELECT
    Status,
    COUNT(*) as Count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims), 2) as Percentage
FROM claims
GROUP BY Status
ORDER BY Count DESC;
"""

result10 = execute_query(query10, "QUERY 10: Claim status distribution")
display_results(result10, "Claims Status Distribution")


QUERY 10: Claim status distribution
--------------------------------------------------

Claims Status Distribution
   Status  Count  Percentage
Completed    339        33.9
Cancelled    336        33.6
  Pending    325        32.5

Total records: 3


In [ ]:
# QUERY 11: Average quantity of food claimed per receiver

query11 = """
SELECT
    r.Name as Receiver_Name,
    r.Type as Receiver_Type,
    r.City,
    COUNT(c.Claim_ID) as Total_Claims,
    COALESCE(AVG(fl.Quantity), 0) as Average_Quantity_Claimed,
    COALESCE(SUM(fl.Quantity), 0) as Total_Quantity_Claimed,
    MIN(fl.Quantity) as Min_Quantity_Claimed,
    MAX(fl.Quantity) as Max_Quantity_Claimed
FROM receivers r
LEFT JOIN claims c ON r.Receiver_ID = c.Receiver_ID
LEFT JOIN food_listings fl ON c.Food_ID = fl.Food_ID
GROUP BY r.Receiver_ID, r.Name, r.Type, r.City
HAVING COUNT(c.Claim_ID) > 0
ORDER BY Average_Quantity_Claimed DESC
LIMIT 15;
"""

result11 = execute_query(query11, "QUERY 11: Average food quantity per receiver")
display_results(result11, "Average Quantity per Receiver")


QUERY 11: Average food quantity per receiver
--------------------------------------------------

Average Quantity per Receiver
     Receiver_Name Receiver_Type               City  Total_Claims  Average_Quantity_Claimed  Total_Quantity_Claimed  Min_Quantity_Claimed  Max_Quantity_Claimed
       Nancy Silva    Individual       Russellville             1                      50.0                      50                    50                    50
        Lisa Pitts       Shelter   Lake Stephenport             1                      50.0                      50                    50                    50
   Daniel Williams       Shelter       Russellburgh             1                      50.0                      50                    50                    50
      Peggy Knight       Shelter North Michaelville             1                      50.0                      50                    50                    50
Christopher Wright       Shelter      New Kellytown             1       

In [ ]:
# QUERY 12: Which meal type is claimed the most?

query12 = """
SELECT
    fl.Meal_Type,
    COUNT(c.Claim_ID) as Total_Claims,
    COUNT(CASE WHEN c.Status = 'Completed' THEN 1 END) as Completed_Claims,
    COUNT(CASE WHEN c.Status = 'Pending' THEN 1 END) as Pending_Claims,
    SUM(fl.Quantity) as Total_Quantity_Claimed,
    AVG(fl.Quantity) as Average_Quantity_Per_Claim,
    ROUND(COUNT(c.Claim_ID) * 100.0 / (SELECT COUNT(*) FROM claims), 2) as Percentage_of_Total_Claims
FROM food_listings fl
JOIN claims c ON fl.Food_ID = c.Food_ID
GROUP BY fl.Meal_Type
ORDER BY Total_Claims DESC;
"""

result12 = execute_query(query12, "QUERY 12: Most claimed meal types")
display_results(result12, "Meal Type Claims Analysis")


QUERY 12: Most claimed meal types
--------------------------------------------------

Meal Type Claims Analysis
Meal_Type  Total_Claims  Completed_Claims  Pending_Claims  Total_Quantity_Claimed  Average_Quantity_Per_Claim  Percentage_of_Total_Claims
Breakfast           278                95              84                    7363                   26.485612                        27.8
    Lunch           250                84              89                    6373                   25.492000                        25.0
   Snacks           240                74              78                    6336                   26.400000                        24.0
   Dinner           232                86              74                    5887                   25.375000                        23.2

Total records: 4


In [ ]:
# QUERY 13: Total quantity of food donated by each provider

query13 = """
SELECT
    p.Name as Provider_Name,
    p.Type as Provider_Type,
    p.City,
    COUNT(fl.Food_ID) as Total_Listings,
    SUM(fl.Quantity) as Total_Food_Donated,
    AVG(fl.Quantity) as Average_Per_Listing,
    COUNT(DISTINCT fl.Food_Type) as Variety_of_Food_Types,
    COUNT(c.Claim_ID) as Total_Claims_Received,
    SUM(CASE WHEN c.Status = 'Completed' THEN fl.Quantity ELSE 0 END) as Food_Successfully_Distributed
FROM providers p
LEFT JOIN food_listings fl ON p.Provider_ID = fl.Provider_ID
LEFT JOIN claims c ON fl.Food_ID = c.Food_ID
GROUP BY p.Provider_ID, p.Name, p.Type, p.City
ORDER BY Total_Food_Donated DESC
LIMIT 20;
"""

result13 = execute_query(query13, "QUERY 13: Food donations by provider")
display_results(result13, "Provider Donation Summary")



QUERY 13: Food donations by provider
--------------------------------------------------

Provider Donation Summary
               Provider_Name    Provider_Type                City  Total_Listings  Total_Food_Donated  Average_Per_Listing  Variety_of_Food_Types  Total_Claims_Received  Food_Successfully_Distributed
           Butler-Richardson    Grocery Store    East Heatherport              13                 365            28.076923                      2                     12                            129
                  Nelson LLC       Restaurant    Lake Andrewmouth               9                 334            37.111111                      3                      9                            123
                 Barry Group       Restaurant       South Kathryn               9                 307            34.111111                      3                      7                            140
               Rogers-Warren    Grocery Store           Jamesfurt               9   

In [ ]:
# QUERY 14: Food wastage analysis - Items close to expiry

query14 = """
SELECT
    CASE
        WHEN DATE(Expiry_Date) < DATE('now') THEN 'Expired'
        WHEN DATE(Expiry_Date) <= DATE('now', '+1 day') THEN 'Expires Today/Tomorrow'
        WHEN DATE(Expiry_Date) <= DATE('now', '+3 days') THEN 'Expires in 2-3 Days'
        WHEN DATE(Expiry_Date) <= DATE('now', '+7 days') THEN 'Expires in 4-7 Days'
        ELSE 'More than 7 Days'
    END as Expiry_Status,
    COUNT(*) as Number_of_Items,
    SUM(Quantity) as Total_Quantity,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM food_listings), 2) as Percentage_of_Items,
    COUNT(c.Claim_ID) as Items_with_Claims,
    COUNT(CASE WHEN c.Status = 'Completed' THEN 1 END) as Items_Successfully_Claimed
FROM food_listings fl
LEFT JOIN claims c ON fl.Food_ID = c.Food_ID
GROUP BY
    CASE
        WHEN DATE(Expiry_Date) < DATE('now') THEN 'Expired'
        WHEN DATE(Expiry_Date) <= DATE('now', '+1 day') THEN 'Expires Today/Tomorrow'
        WHEN DATE(Expiry_Date) <= DATE('now', '+3 days') THEN 'Expires in 2-3 Days'
        WHEN DATE(Expiry_Date) <= DATE('now', '+7 days') THEN 'Expires in 4-7 Days'
        ELSE 'More than 7 Days'
    END
ORDER BY
    CASE Expiry_Status
        WHEN 'Expired' THEN 1
        WHEN 'Expires Today/Tomorrow' THEN 2
        WHEN 'Expires in 2-3 Days' THEN 3
        WHEN 'Expires in 4-7 Days' THEN 4
        ELSE 5
    END;
"""

result14 = execute_query(query14, "QUERY 14: Food wastage analysis by expiry dates")
display_results(result14, "Food Expiry Analysis")


QUERY 14: Food wastage analysis by expiry dates
--------------------------------------------------

Food Expiry Analysis
   Expiry_Status  Number_of_Items  Total_Quantity  Percentage_of_Items  Items_with_Claims  Items_Successfully_Claimed
More than 7 Days             1353           35119                135.3               1000                         339

Total records: 1


In [ ]:
# QUERY 15: Comprehensive system performance metrics

query15 = """
SELECT
    'System Overview' as Metric_Category,
    'Total Providers' as Metric_Name,
    COUNT(*) as Value,
    NULL as Percentage
FROM providers

UNION ALL

SELECT
    'System Overview' as Metric_Category,
    'Total Receivers' as Metric_Name,
    COUNT(*) as Value,
    NULL as Percentage
FROM receivers

UNION ALL

SELECT
    'Food Listings' as Metric_Category,
    'Total Food Items Listed' as Metric_Name,
    COUNT(*) as Value,
    NULL as Percentage
FROM food_listings

UNION ALL

SELECT
    'Food Listings' as Metric_Category,
    'Total Food Quantity Available' as Metric_Name,
    SUM(Quantity) as Value,
    NULL as Percentage
FROM food_listings

UNION ALL

SELECT
    'Claims Performance' as Metric_Category,
    'Total Claims Made' as Metric_Name,
    COUNT(*) as Value,
    NULL as Percentage
FROM claims

UNION ALL

SELECT
    'Claims Performance' as Metric_Category,
    'Successful Claims' as Metric_Name,
    COUNT(*) as Value,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims), 2) as Percentage
FROM claims
WHERE Status = 'Completed'

UNION ALL

SELECT
    'Claims Performance' as Metric_Category,
    'Pending Claims' as Metric_Name,
    COUNT(*) as Value,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims), 2) as Percentage
FROM claims
WHERE Status = 'Pending'

UNION ALL

SELECT
    'Efficiency Metrics' as Metric_Category,
    'Items with Claims' as Metric_Name,
    COUNT(DISTINCT Food_ID) as Value,
    ROUND(COUNT(DISTINCT Food_ID) * 100.0 / (SELECT COUNT(*) FROM food_listings), 2) as Percentage
FROM claims

UNION ALL

SELECT
    'Food Impact' as Metric_Category,
    'Food Successfully Distributed' as Metric_Name,
    SUM(fl.Quantity) as Value,
    ROUND(SUM(fl.Quantity) * 100.0 / (SELECT SUM(Quantity) FROM food_listings), 2) as Percentage
FROM food_listings fl
JOIN claims c ON fl.Food_ID = c.Food_ID
WHERE c.Status = 'Completed'

ORDER BY Metric_Category, Metric_Name;
"""

result15 = execute_query(query15, "QUERY 15: Comprehensive system performance metrics")
display_results(result15, "System Performance Dashboard")


QUERY 15: Comprehensive system performance metrics
--------------------------------------------------

System Performance Dashboard
   Metric_Category                   Metric_Name  Value  Percentage
Claims Performance                Pending Claims    325       32.50
Claims Performance             Successful Claims    339       33.90
Claims Performance             Total Claims Made   1000         NaN
Efficiency Metrics             Items with Claims    647       64.70
       Food Impact Food Successfully Distributed   8773       34.01
     Food Listings       Total Food Items Listed   1000         NaN
     Food Listings Total Food Quantity Available  25794         NaN
   System Overview               Total Providers   1000         NaN
   System Overview               Total Receivers   1000         NaN

Total records: 9


In [ ]:
print("\n" + "="*80)
print("FOOD DONATION SYSTEM - ANALYTICAL SUMMARY REPORT")
print("="*80)

# Key Insights
print("\n KEY INSIGHTS:")
print("-" * 40)

if not result5.empty:
    total_food = result5['Total_Food_Available'].iloc[0]
    total_listings = result5['Total_Food_Listings'].iloc[0]
    print(f"• Total food available: {total_food:,} units across {total_listings} listings")

if not result10.empty:
    for _, row in result10.iterrows():
        print(f"• {row['Status']} claims: {row['Count']} ({row['Percentage']}%)")

if not result6.empty and len(result6) > 0:
    top_city = result6.iloc[0]
    print(f"• Most active city: {top_city['City']} with {top_city['Food_Listings']} listings")

if not result2.empty and len(result2) > 0:
    top_provider_type = result2.iloc[0]
    print(f"• Top provider type: {top_provider_type['Provider_Type']} with {top_provider_type['Total_Food_Quantity']} units")

print("\n RECOMMENDATIONS:")
print("-" * 40)

# Check for expired items
if not result14.empty:
    expired_items = result14[result14['Expiry_Status'] == 'Expired']
    if len(expired_items) > 0:
        print(f"• URGENT: {expired_items['Number_of_Items'].iloc[0]} items have expired - review expiry management")

    expiring_soon = result14[result14['Expiry_Status'].str.contains('Today|Tomorrow')]
    if len(expiring_soon) > 0:
        print(f"• {expiring_soon['Number_of_Items'].iloc[0]} items expiring soon - prioritize these for claims")

# Check claim completion rate
if not result10.empty:
    completed_claims = result10[result10['Status'] == 'Completed']
    if len(completed_claims) > 0 and completed_claims['Percentage'].iloc[0] < 70:
        print("• Consider improving claim completion process - success rate below 70%")

print("\nANALYSIS COMPLETE!")
print("All 15 SQL queries executed successfully.")

# Close database connection
if conn:
    conn.close()
    print(" Database connection closed.")

print("\n Analysis completed! Review the results above for comprehensive insights into your food donation system.")


FOOD DONATION SYSTEM - ANALYTICAL SUMMARY REPORT

 KEY INSIGHTS:
----------------------------------------
• Total food available: 25,794 units across 1000 listings
• Completed claims: 339 (33.9%)
• Cancelled claims: 336 (33.6%)
• Pending claims: 325 (32.5%)
• Most active city: South Kathryn with 6 listings
• Top provider type: Restaurant with 6923 units

 RECOMMENDATIONS:
----------------------------------------
• Consider improving claim completion process - success rate below 70%

ANALYSIS COMPLETE!
All 15 SQL queries executed successfully.
 Database connection closed.

 Analysis completed! Review the results above for comprehensive insights into your food donation system.
